# Working with DataFrames

Parsing, cleaning, creating DFs

In [1]:
import pandas as pd
import config as cf

FileNotFoundError: [Errno 2] No such file or directory: 'data/major_version_1.tsv'

In [ ]:
v2 = cf.get_v2()
v3 = cf.get_v3()
spark = cf.get_spark_interfax()

## Cleaning Major Tables

#### V3 Major DF

In [ ]:
print(" \nCount total NaN at each column in a DataFrame : \n\n", v2.isnull().sum())
v3.shape

In [ ]:
v3 = v3.drop('Unnamed: 0', axis = 1)
v3 = v3.rename(columns = {'tone': 'Tone'})
v3 = v3.sort_values(by='Company')
v3 = v3.reset_index(drop=True)

v3.head(1)

In [ ]:
# Define a dictionary to map the numbers to their corresponding labels
label_mapping = {0: 'leaved', 1: 'continue', 2: 'paused'}

# Replace the numbers in the specified column with the labels
v3['Tone'] = v3['Tone'].map(label_mapping)

v3.head(10)

#### V2 Major DF

In [ ]:
v2 = v2.rename(columns={'i': 'Company'})
v2 = v2.rename(columns={'ИНН': 'INT'})
v2 = v2.sort_values(by='Company')
v2 = v2.reset_index(drop=True)

v2.head(1)

#### Creating Merged DF

Merge the two datasets based on the "Company" column

In [ ]:
merged = pd.merge(v3, v2, on='Company')

merged.head(1)

Select the desired columns

In [ ]:
desired_columns = ['Company', 'Action_x', 'Industry_x', 'Country_x', 'Tone', 'INT']
merged = merged[desired_columns]

merged.head(1)

Rename the columns

In [ ]:
merged.columns = ['Company', 'Action', 'Industry', 'Country', 'Tone', 'INT']

merged.head(1)

## Cleaning Component Tables (C-Tables)

#### Spark Interfax DF

In [ ]:
spark.head()

In [ ]:
spark.shape

In [ ]:
columns_to_drop = ['№', 'Наименование', 'Регистрационный номер', 'Краткое наименование', 'Дата регистрации']
spark = spark.drop(columns_to_drop, axis=1)

print(spark.columns)

Rename columns 'Наименование на английском' and 'Код налогоплательщика' to 'Company' and 'INT'

In [ ]:
spark = spark.rename(columns={"Наименование на английском": "Company"})
spark = spark.rename(columns={"Код налогоплательщика": "INT"})
# spark['Company'] = spark['Company'].str.lower()

spark.head(1)

## Creating Main Working DF (Majors + C-Tables)

In [ ]:
df = merged.merge(spark, on='INT')

df.head(10)

In [ ]:
df = df.rename(columns={"Company_y": "Company"})

df = df.drop({'Company_x', 'INT',
                            '2017, Среднесписочная численность работников',
                            '2018, Среднесписочная численность работников',
                            '2019, Среднесписочная численность работников',
                            '2020, Среднесписочная численность работников',
                            '2021, Среднесписочная численность работников',
                            'Мои списки'}, axis=1)

df = df[['Company', 'Industry', 'Country', 'Action', 'Tone', 'Возраст компании, лет',
                       'Вид деятельности/отрасль', 'Организационно-правовая форма',
                       'Форма собственности', 'Размер компании', '2017, Выручка, RUB',
                       '2018, Выручка, RUB', '2019, Выручка, RUB', '2020, Выручка, RUB',
                       '2021, Выручка, RUB', '2017, Прибыль (убыток) до налогообложения , RUB',
                       '2018, Прибыль (убыток) до налогообложения , RUB',
                       '2019, Прибыль (убыток) до налогообложения , RUB',
                       '2020, Прибыль (убыток) до налогообложения , RUB',
                       '2021, Прибыль (убыток) до налогообложения , RUB']]

df.head(10)

The column 'Возраст компании, лет' is of string datatype. To make it float datatype, we replace , with .

In [ ]:
if df['Возраст компании, лет'].dtype == object:
    # Replace commas with periods in the column
    df['Возраст компании, лет'] = df['Возраст компании, лет'].str.replace(',', '.')
df['Возраст компании, лет'] = df['Возраст компании, лет'].astype(float)

Some columns has big number with spaces, therefore they are of a string type.  To make them float datatype, we delete all unnecassary spaces

In [ ]:
df['2017, Выручка, RUB'] = df['2017, Выручка, RUB'].str.replace(' ', '').astype(float)
df['2018, Выручка, RUB'] = df['2018, Выручка, RUB'].str.replace(' ', '').astype(float)
df['2019, Выручка, RUB'] = df['2019, Выручка, RUB'].str.replace(' ', '').astype(float)
df['2020, Выручка, RUB'] = df['2020, Выручка, RUB'].str.replace(' ', '').astype(float)
df['2021, Выручка, RUB'] = df['2021, Выручка, RUB'].str.replace(' ', '').astype(float)
df['2017, Прибыль (убыток) до налогообложения , RUB'] = df['2017, Прибыль (убыток) до налогообложения , RUB'].str.replace(' ', '').astype(float)
df['2018, Прибыль (убыток) до налогообложения , RUB'] = df['2018, Прибыль (убыток) до налогообложения , RUB'].str.replace(' ', '').astype(float)
df['2019, Прибыль (убыток) до налогообложения , RUB'] = df['2019, Прибыль (убыток) до налогообложения , RUB'].str.replace(' ', '').astype(float)
df['2020, Прибыль (убыток) до налогообложения , RUB'] = df['2020, Прибыль (убыток) до налогообложения , RUB'].str.replace(' ', '').astype(float)
df['2021, Прибыль (убыток) до налогообложения , RUB'] = df['2021, Прибыль (убыток) до налогообложения , RUB'].str.replace(' ', '').astype(float)

df.head(10)

In [ ]:
df.shape

## Handling missing values in DF

Count total NaN at each column in a database

In [ ]:
print(" \nCount total NaN at each column in a DataFrame : \n\n", df.isnull().sum())

Visualize missing values in dataset

In [ ]:
import missingno as msno
msno.bar(df)

In [ ]:
msno.heatmap(df)

Filling NaN profit values with its mean. Delete all rows where more than 6 NaN values exist.

In [ ]:
df.dropna(thresh=df.shape[1] - 6, inplace=True)
print(" \nCount total NaN at each column in a DataFrame : \n\n", df.isnull().sum())
df.shape

Delete all rows where Nan values exist.

In [ ]:
df['Mean Revenue'] = df[['2017, Выручка, RUB', '2018, Выручка, RUB', '2019, Выручка, RUB', '2020, Выручка, RUB', '2021, Выручка, RUB']].mean(axis=1)
df.dropna(subset=['Mean Revenue'], inplace=True)
df.drop(['2017, Выручка, RUB', '2018, Выручка, RUB', '2019, Выручка, RUB', '2020, Выручка, RUB', '2021, Выручка, RUB'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
print(" \nCount total NaN at each column in a DataFrame : \n\n", df.isnull().sum())

In [ ]:
df['Mean Profit'] = df[['2017, Прибыль (убыток) до налогообложения , RUB', '2018, Прибыль (убыток) до налогообложения , RUB', '2019, Прибыль (убыток) до налогообложения , RUB', '2020, Прибыль (убыток) до налогообложения , RUB', '2021, Прибыль (убыток) до налогообложения , RUB']].mean(axis=1)
df.dropna(subset=['Mean Revenue'], inplace=True)
df.drop(['2017, Прибыль (убыток) до налогообложения , RUB', '2018, Прибыль (убыток) до налогообложения , RUB', '2019, Прибыль (убыток) до налогообложения , RUB', '2020, Прибыль (убыток) до налогообложения , RUB', '2021, Прибыль (убыток) до налогообложения , RUB'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
print(" \nCount total NaN at each column in a DataFrame : \n\n", database.isnull().sum())

In [ ]:
df.dropna(inplace=True)
print(" \nCount total NaN at each column in a DataFrame : \n\n", database.isnull().sum())

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.corr()

Now our dataset has no NaN values and we can start clusterisation!